In [1]:
import time
import numpy as np
from scipy.constants import elementary_charge, proton_mass 
from datetime import date
from datetime import datetime
import os
import vis_canonical_fluxtubes as vis

In [2]:
import visit
visit.Launch()

True

In [3]:
database_prefix = '/Users/vonderlinden2/rsx_analysis/writing_to_vtk/output/2016-08-14-17-44'

In [4]:
today = str(date.today())
out_dir = '../output/' + today
try:
    os.makedirs(out_dir)
except:
    pass

visit.OpenDatabase(database_prefix + '/Bdot_triple_probe_quantities*.vtk database')
field_nulls = np.loadtxt('../../centroid_fitting/output/2016-08-12/field_nulls.txt')
electron_points_outer, electron_points_inner = vis.launch_points(field_nulls[0], num_inner=10)
ion_points_outer, ion_points_inner = vis.launch_points(field_nulls[0], plane=0.249, num_inner=80)
vis.define_expressions(visit)
vis.setup_current_pseudocolor(visit, max_val=5.1e5, min_val=0.0)
(electron_StreamlineAtts_outer, 
 electron_StreamlineAtts_inner) = vis.setup_massless_electron_canonical_flux_tubes(visit, electron_points_inner, 
                                                                                   electron_points_inner)
(ion_StreamlineAtts_forward, 
 ion_StreamlineAtts_backward) = vis.setup_forward_backward_ion_canonical_flux_tubes(visit, ion_points_inner, 
                                                                                    ion_points_inner,
                                                                                    forward_color=vis.olive,
                                                                                    backward_color=vis.olive)
AnnotationAtts = vis.setup_annotations(visit, time_scale=0.068)
vis.set_default_view(visit)
visit.DrawPlots()

1L

In [5]:
visit.OpenGUI()

1L

In [6]:
output_path = out_dir + '/ion_canonical_flux_tube'
ending = '.png'
plane = 0.249
save_atts = visit.SaveWindowAttributes()
save_atts.format = save_atts.PNG
save_atts.height = 1080
save_atts.width = 1920
save_atts.family = 0
visit.SetSaveWindowAttributes(save_atts)

for time_point in xrange(0, 206):
    print time_point
    save_atts.fileName = output_path + str(time_point).zfill(4) + ending
    visit.SetSaveWindowAttributes(save_atts)
    
    points_outer, points_inner = vis.launch_points(field_nulls[time_point], plane)
    e_points_outer, e_points_inner = vis.launch_points(field_nulls[time_point], plane, num_inner=10)
    
    visit.SetActivePlots(1)
    electron_StreamlineAtts_outer.SetPointList(e_points_inner)
    visit.SetPlotOptions(electron_StreamlineAtts_outer)
    
    visit.SetActivePlots(2)
    electron_StreamlineAtts_inner.SetPointList(e_points_inner)
    visit.SetPlotOptions(electron_StreamlineAtts_inner)
    
    visit.SetActivePlots(3)
    ion_StreamlineAtts_forward.SetPointList(points_inner)
    visit.SetPlotOptions(ion_StreamlineAtts_forward)
    
    visit.SetActivePlots(4)
    ion_StreamlineAtts_backward.SetPointList(points_inner)
    visit.SetPlotOptions(ion_StreamlineAtts_backward)
    
    visit.SetTimeSliderState(time_point)
    
    name = visit.SaveWindow()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147


KeyboardInterrupt: 

In [8]:
save_atts = visit.SaveWindowAttributes()
save_atts.format = save_atts.PNG
save_atts.height = 1080
save_atts.width = 1920
save_atts.family = 0
visit.SetSaveWindowAttributes(save_atts)
ending='.png'

for time_point in xrange(226, 250):
    print time_point
    save_atts.fileName = output_path + str(time_point - 20).zfill(4) + ending
    visit.SetSaveWindowAttributes(save_atts)
    
    points_outer, points_inner = vis.launch_points(field_nulls[time_point], plane)
    e_points_outer, e_points_inner = vis.launch_points(field_nulls[time_point], plane, num_inner=10)
    
    visit.SetActivePlots(1)
    electron_StreamlineAtts_outer.SetPointList(e_points_inner)
    visit.SetPlotOptions(electron_StreamlineAtts_outer)
    
    visit.SetActivePlots(2)
    electron_StreamlineAtts_inner.SetPointList(e_points_inner)
    visit.SetPlotOptions(electron_StreamlineAtts_inner)
    
    visit.SetActivePlots(3)
    ion_StreamlineAtts_forward.SetPointList(points_inner)
    visit.SetPlotOptions(ion_StreamlineAtts_forward)
    
    visit.SetActivePlots(4)
    ion_StreamlineAtts_backward.SetPointList(points_inner)
    visit.SetPlotOptions(ion_StreamlineAtts_backward)
    
    visit.SetTimeSliderState(time_point)
    
    name = visit.SaveWindow()

226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249


In [11]:
!ffmpeg -framerate 30 -i ../output/2016-08-21/ion_canonical_flux_tube%04d.png -r 30 -pix_fmt yuv420p \
 -vcodec libx264 -vf "scale=trunc(iw/2)*2:trunc(ih/2)*2" -y ../output/2016-08-21/ion_canonical_fluxtube_evolution.mp4

ffmpeg version 2.8.1 Copyright (c) 2000-2015 the FFmpeg developers
  built with Apple LLVM version 6.0 (clang-600.0.56) (based on LLVM 3.5svn)
  configuration: --prefix=/Users/vonderlinden2/homebrew/Cellar/ffmpeg/2.8.1_1 --enable-shared --enable-pthreads --enable-gpl --enable-version3 --enable-hardcoded-tables --enable-avresample --cc=clang --host-cflags= --host-ldflags= --enable-opencl --enable-libx264 --enable-libmp3lame --enable-libvo-aacenc --enable-libxvid --enable-vda
  libavutil      54. 31.100 / 54. 31.100
  libavcodec     56. 60.100 / 56. 60.100
  libavformat    56. 40.101 / 56. 40.101
  libavdevice    56.  4.100 / 56.  4.100
  libavfilter     5. 40.101 /  5. 40.101
  libavresample   2.  1.  0 /  2.  1.  0
  libswscale      3.  1.101 /  3.  1.101
  libswresample   1.  2.101 /  1.  2.101
  libpostproc    53.  3.100 / 53.  3.100
Input #0, image2, from '../output/2016-08-21/ion_canonical_flux_tube%04d.png':
  Duration: 00:00:07.67, start: 0.000000, bitrate: N/A
    Stream #0:0: V